In [10]:
! pip install ipynb
! pip install torch

In [11]:
import ipynb.fs.full.basics_laplace as basics_laplace
import torch

## Implementing a 1-hop GCN layer in Pytorch

Y is the output. Lnorm is the Laplacian of the Adjacency Matrix A. X is the graph signal. W are the learnable parameters. D is the Degree-Matrix of A. I is the Identity Matrix.

$$Y=Lnorm​XW$$
$$L^{norm}mod​=D^{−1/2}​(A+I)D^{-1/2}​$$

In [15]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

def device_as(x,y):
    return x.to(y.device)

def calc_degree_matrix_norm(a):
    return torch.diag_embed(torch.pow(a.sum(dim=-1),-0.5))

def create_graph_lapl_norm(a):
    size = a.shape[-1]
    a +=  device_as(torch.eye(size),a)
    D_norm = calc_degree_matrix_norm(a)
    L_norm = torch.bmm( torch.bmm(D_norm, a) , D_norm )
    return L_norm

class GCN_AISUMMER(nn.Module):
    """
    A simple GCN layer, similar to https://arxiv.org/abs/1609.02907
    """
    def __init__(self, in_features, out_features, bias=True):
        super().__init__()
        self.linear = nn.Linear(in_features, out_features, bias=bias)

    def forward(self, X, A):
        """
        A: adjecency matrix
        X: graph signal
        """
        L = create_graph_lapl_norm(A)
        x = self.linear(X)
        return torch.bmm(L, x)

In [16]:
import torch
from torch import nn
import torch.nn.functional as F

class GNN(nn.Module):
    def __init__(self,
                    in_features = 7,
                    hidden_dim = 64,
                    classes = 2,
                    dropout = 0.5):
        super(GNN, self).__init__()

        self.conv1 = GCN_AISUMMER(in_features, hidden_dim)
        self.conv2 = GCN_AISUMMER(hidden_dim, hidden_dim)
        self.conv3 = GCN_AISUMMER(hidden_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, classes)
        self.dropout = dropout

    def forward(self, x,A):
        x = self.conv1(x, A)
        x = F.relu(x)
        x = self.conv2(x, A)
        x = F.relu(x)
        x = self.conv3(x, A)
        x = F.dropout(x, p=self.dropout, training=self.training)
        # aggregate node embeddings
        x = x.mean(dim=1)
        # final classification layer
        return self.fc(x)